# Length Of Stay COVID-19 Patients

### documentation:  

* forsight: https://pages.github.cerner.com/foresight/foresight/machine_learning.html
* spark mllib:  https://spark.apache.org/docs/latest/ml-guide.html
* spark sql built-in functions: https://spark.apache.org/docs/latest/api/sql/
* spark sql:  https://spark.apache.org/docs/latest/sql-programming-guide.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636778915296_0298'

In [3]:
pd.options.display.max_columns = 35

In [4]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831


In [5]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


### Create Variables for Queries

In [6]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


In [7]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [8]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [9]:
spark.sql(f"""select * from {userName}.{algorithmName}_Hospital h""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- intensive_care_visit: integer (nullable = true)
 |-- intensive_care_medical_service: integer (nullable = true)
 |-- palliative_care_visit: integer (nullab

In [10]:
spark.sql(f"""select * from {userName}.{algorithmName}_Hospital h""").limit(5).toPandas()

,tenant,tenant_bed_size,tenant_speciality,tenant_segment,tenant_zip_code,personId,encounterId,serviceDate,dischargeDate,length_of_stay,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,hospice_care_medical_service,age,gender
0,1,500-999,IDN/Regional Health Authority,Regional Hospital,5,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,2020-10-25T03:56:55+00:00,2020-10-28T21:35:34+00:00,4,5,0,0,0,0,0,0,0,0,0,0,0,0,61.0,female
1,63,>=1000,IDN/Regional Health Authority,IDN,8,018f1713-c7ec-4254-bb07-8f810835d817,fea7c698-8219-4561-bf1c-7a6f21a84e5b,2021-04-11T12:57:00+00:00,2021-04-30T19:36:00+00:00,20,8,0,0,0,0,0,0,0,0,0,0,0,0,43.0,female
2,60,500-999,IDN/Regional Health Authority,Regional Hospital,5,0224edb3-9d4a-4bbd-8850-e9ad780c3764,b132f569-10d7-487c-baf8-91dae774e91a,2020-07-13T15:13:46+00:00,2020-07-22T23:51:00+00:00,10,5,0,0,0,0,0,0,0,0,0,0,0,0,40.0,male
3,45,>=1000,IDN/Regional Health Authority,IDN,1,02a16121-ebaf-40d8-aa5b-e107dad91ff9,84887042-d69d-49d9-a05c-7ec52b3c4bf6,2020-12-01T18:13:00+00:00,2020-12-06T21:05:00+00:00,6,8,0,1,0,0,0,0,0,0,0,0,0,0,68.0,female
4,13,>=1000,IDN/Regional Health Authority,IDN,3,02cbb97a-3983-4f03-8628-cdb715880bb5,a1dbb021-9477-48b4-a802-d454d6a9dcdb,2020-10-25T02:26:00+00:00,2020-10-25T23:00:00+00:00,1,3,0,0,0,0,0,0,0,0,0,0,0,0,22.0,male


In [11]:
spark.sql(f""" select distinct tenant_bed_size from {userName}.{algorithmName}_Hospital h """).toPandas()

,tenant_bed_size
0,<100
1,200-299
2,100-199
3,500-999
4,>=1000
5,300-499


In [12]:
spark.sql(f""" select distinct gender from {userName}.{algorithmName}_Hospital h """).toPandas()

,gender
0,unknown
1,female
2,male


In [13]:
hospitalDf = spark.sql(f"""
  select tenant
         ,personId
         ,encounterId
         ,length_of_stay
         ,case
                when tenant_bed_size = "<100" then 1
                when tenant_bed_size = "100-299" then 2
                when tenant_bed_size = "200-299" then 3
                when tenant_bed_size = "300-499" then 3
                when tenant_bed_size = "500-999" then 4
                when tenant_bed_size = ">=1000" then 5
              else 0
              end as bed_size_index
        ,case
               when discharged_to_skilled_nursing_facility = 1 then 1
               when discharged_to_hospice = 1 then 2
               when died_in_the_hospital = 1 then 3
               when discharged_to_rehab_facility = 1 then 4
               when discharged_to_long_term_care_facility = 1 then 5
              else 0
              end as discharge_to_index     
         ,case
                when age >= 90 then 90
                when age >= 85 and age < 90 then 85
                when age >= 80 and age < 85 then 80
                when age >= 75 and age < 80 then 75
                when age >= 70 and age < 75 then 70
                when age >= 65 and age < 70 then 65
                when age >= 60 and age < 65 then 60
                when age >= 55 and age < 60 then 55
                when age >= 50 and age < 55 then 50
                when age >= 45 and age < 50 then 45
                when age >= 40 and age < 45 then 40
                when age >= 35 and age < 40 then 35
                when age >= 30 and age < 35 then 30
                when age >= 25 and age < 30 then 25
                when age >= 20 and age < 25 then 20
                when age >= 12 and age < 20 then 12
                when age >= 0 and age < 12 then 1
              else 0
              end as age_range
         ,if(tenant_zip_code is null, 99, tenant_zip_code) as tenant_zip_code  
         ,if(zip_code is null, 99, tenant_zip_code) as zip_code 
         ,deceased
         ,discharged_to_skilled_nursing_facility
         ,discharged_to_hospice
         ,died_in_the_hospital
         ,discharged_to_rehab_facility 
         ,discharged_to_long_term_care_facility
         ,intensive_care_visit 
         ,intensive_care_medical_service
         ,palliative_care_visit
         ,palliative_care_medical_service
         ,hospice_care_visit
         ,hospice_care_medical_service
         ,case
                when gender = "male" then 1
                when gender = "female" then 2
              else 0
              end as gender_index
        
         from {userName}.{algorithmName}_Hospital h 
         
         order by 1

""")

In [14]:
hospitalDf.toPandas().head()

,tenant,personId,encounterId,length_of_stay,bed_size_index,discharge_to_index,age_range,tenant_zip_code,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,hospice_care_medical_service,gender_index
0,1,00a50ac0-9c00-43ba-9d25-0dcc0d23549f,757500da-0380-496f-9712-32eebe6004d7,50,4,0,30,5,5,0,0,0,0,0,0,0,0,0,0,0,0,2
1,1,0ec27890-8768-495a-811f-4dd73fa7344f,d5750f34-7d9c-4f59-9b23-ecf20aab5552,3,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,29a2d106-c71e-47b0-92cb-89d70ce00c6b,2e6a2779-937e-4238-adca-b0ed97d18dea,3,4,0,65,5,5,0,0,0,0,0,0,0,0,0,0,0,0,2
3,1,433332f0-775b-49ae-b1d4-b03979c1894a,c96c123c-d9c5-4302-9335-de943a33ebd7,10,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,64d355a5-5910-4682-894e-6620d2786926,eafca448-6ebb-4c2a-a419-26995aa4c8a2,2,4,0,70,5,5,0,0,0,0,0,0,0,0,0,0,0,0,1


In [15]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_hospital_ml""")

DataFrame[]

In [16]:
hospitalDf.write.saveAsTable(f"""{userName}.{algorithmName}_hospital_ml""")

In [33]:
spark.sql(f"""select * from {userName}.{algorithmName}_hospital_ml""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- bed_size_index: integer (nullable = true)
 |-- discharge_to_index: integer (nullable = true)
 |-- age_range: integer (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- intensive_care_visit: integer (nullable = true)
 |-- intensive_care_medical_service: integer (nullable = true)
 |-- palliative_care_visit: integer (nullable = true)
 |-- palliative_care_medical_service: integer (nullable = true)
 |-- hospice_ca

In [17]:
spark.sql(f"""select * from {userName}.{algorithmName}_Vaccine_Detail""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- encounterType: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- vaccine_date: string (nullable = true)
 |-- vaccinated: integer (nullable = true)
 |-- taken_Moderna_vaccine: integer (nullable = true)
 |-- taken_Pfizer_vaccine: integer (nullable = true)
 |-- taken_AstraZeneca_vaccine: integer (nullable = true)
 |-- taken_Janssen_vaccine: integer (nullable = true)
 |-- taken_Unspecified_vaccine: integer (nullable = true)
 |-- taken_Non_US_vaccine: integer (nullable = true)
 |-- taken_single_dose: integer (nullable = true)
 |-- taken_multiple_doses: integer (nullable = true)



In [18]:
spark.sql(f"""select * from {userName}.{algorithmName}_Vaccine_Detail""").limit(5).toPandas()

,tenant,personId,encounterId,encounterType,serviceDate,vaccine_date,vaccinated,taken_Moderna_vaccine,taken_Pfizer_vaccine,taken_AstraZeneca_vaccine,taken_Janssen_vaccine,taken_Unspecified_vaccine,taken_Non_US_vaccine,taken_single_dose,taken_multiple_doses
0,33,00afdba6-b8d0-469d-97a1-988b5423626b,49dd2b05-24a3-4365-89bc-2cd688e8252c,Seen by nurse,2021-02-10T13:55:05+00:00,2021-02-10T14:00:00+00:00,1,0,1,0,0,0,0,0,0
1,37,06367269-b31f-41ff-9b3c-ab4dc2fea354,06b48e93-bdd0-4823-a429-5270457e4dfe,None,2021-01-29T15:55:54+00:00,2021-01-29T16:00:00+00:00,1,0,1,0,0,0,0,0,0
2,37,06367269-b31f-41ff-9b3c-ab4dc2fea354,fe06d005-c88f-41df-ac7e-7a71edcbb8ba,None,2021-01-06T15:54:32+00:00,2021-01-06T16:00:00+00:00,1,0,1,0,0,0,0,0,0
3,63,0b7548fd-894d-4dbd-be5a-e8f02ff036d9,d63fd056-a20f-4afa-8b9d-4b2a3051a927,Attending clinic,2021-03-02T15:50:51+00:00,2021-03-02T16:08:00+00:00,1,0,1,0,0,0,0,0,0
4,81,0d190c33-ce80-41d2-9bb2-db50e5bbd948,78508c8b-6072-44e7-8c17-3417d907c2f2,None,2021-05-07T14:24:10+00:00,2021-05-07T15:00:00+00:00,1,0,1,0,0,0,0,0,0


In [30]:
vaccineDf = spark.sql(f"""

      select  h.tenant
             ,h.personId
             ,h.encounterId
             ,if(vaccinated is null, 0, vaccinated) as vaccinated
             ,if(taken_Moderna_vaccine is null, 0, taken_Moderna_vaccine) as taken_Moderna_vaccine
             ,if(taken_Pfizer_vaccine is null, 0, taken_Pfizer_vaccine) as taken_Pfizer_vaccine
             ,if(taken_Janssen_vaccine is null, 0, taken_Janssen_vaccine) as taken_Janssen_vaccine
             ,if(taken_Unspecified_vaccine is null, 0, taken_Unspecified_vaccine) as taken_Unspecified_vaccine
             ,if(taken_Non_US_vaccine is null, 0, taken_Non_US_vaccine) as taken_Non_US_vaccine
             ,if(taken_single_dose is null, 0, taken_single_dose) as taken_single_dose
             ,if(taken_multiple_doses is null, 0, taken_multiple_doses) as taken_multiple_doses

        from {userName}.{algorithmName}_hospital h
        
        left join {userName}.{algorithmName}_Vaccine_Detail v
           on ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
              and h.tenant = v.tenant
              and h.personId = v.personId
        
   order by 1, 2, 3
""")

In [31]:
vaccineDf.limit(16).toPandas()

,tenant,personId,encounterId,vaccinated,taken_Moderna_vaccine,taken_Pfizer_vaccine,taken_Janssen_vaccine,taken_Unspecified_vaccine,taken_Non_US_vaccine,taken_single_dose,taken_multiple_doses
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,0,0,0,0,0,0,0,0
1,1,002756bd-6290-4982-9c69-f879158503ab,32d82cb7-69b8-4d9d-ab2d-b97e14c59927,0,0,0,0,0,0,0,0
2,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,0,0,0,0,0,0,0,0
3,1,004f0a70-f538-4e7a-b426-3b7306a59a80,ee12825a-d8a3-4e64-8de6-cd5228c8c16f,0,0,0,0,0,0,0,0
4,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,0,0,0,0,0,0,0,0
5,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,6a06b5e8-cdd5-4e98-84f4-1238237dcc8e,0,0,0,0,0,0,0,0
6,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,aa061342-b172-470d-84b0-f9fe41f63beb,0,0,0,0,0,0,0,0
7,1,00a50ac0-9c00-43ba-9d25-0dcc0d23549f,757500da-0380-496f-9712-32eebe6004d7,0,0,0,0,0,0,0,0
8,1,00a8ceb1-b0bc-40ef-9de7-6e88dc900274,ac4062df-18a5-44b1-b307-ec4bfb5908a9,0,0,0,0,0,0,0,0
9,1,00c9804e-438c-4a7c-95b3-c17718e5b4cf,29a1e91d-468f-46a9-b769-6c03b15f3c25,0,0,0,0,0,0,0,0


In [52]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_vaccine_ml""")

DataFrame[]

In [53]:
vaccineDf.write.saveAsTable(f"""{userName}.{algorithmName}_vaccine_ml""")

In [22]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsCoronavirusLabs""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id: string (nullable = true)
 |-- taken_COVID_19_EVALUATION_NOTE_lab_id: string (nullable = true)
 |-- taken_COVID_19_INITIAL_EVALUATION_FORM_lab_id: string (nullable = true)
 |-- taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_id: string (nullable = true)
 |-- taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_id: string (nullable = true)
 |-- taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_id: string (nullable = true)
 |-- taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_id: string (nullable = true)
 |-- taken_INFLUENZA_VIRUS_A_B_

In [23]:
coronavirusLabsDf = spark.sql(f"""
   select tenant
         ,personId
         ,encounterId
,max(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_COVID_19_EVALUATION_NOTE_lab_test) as taken_COVID_19_EVALUATION_NOTE_lab_test
,max(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_test) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_test
,max(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_test) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_test
,max(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_test) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_test
,max(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_test) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_test) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_test
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_test) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_test) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_test
,max(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_test) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_test
,max(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_test) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_test) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_test
,max(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_test) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_test
,max(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_test) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_test
,max(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_test) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_test) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_test
,max(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_test) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_test) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_test
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_test) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_test) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_test
,max(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
,max(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
,max(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
,max(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_test
,max(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_test) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
,max(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_test) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_test
,max(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_test) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_test
,max(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_test) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_test
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_test) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_test
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_test) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_test
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_test) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_test
,max(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
,max(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
,max(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
,max(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
,max(taken_COVID_19_EVALUATION_NOTE_lab_value) as taken_COVID_19_EVALUATION_NOTE_lab_value
,max(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value
,max(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value
,max(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value
,max(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value
,max(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value
,max(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value
,max(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value
,max(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value
,max(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value
,max(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value
,max(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
,max(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value
,max(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
,max(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
,max(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
,max(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value
,max(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
,max(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value
,max(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value
,max(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value
,max(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value
,max(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
,max(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
,max(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
,max(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
,max(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
   
  from
        (  select cl.tenant
                     ,cl.personId
                     ,cl.encounterId
         ,taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_COVID_19_EVALUATION_NOTE_lab_test
         ,taken_COVID_19_INITIAL_EVALUATION_FORM_lab_test
         ,taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_test
         ,taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_test
         ,taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_test
         ,taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_test
         ,taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_test
         ,taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_test
         ,taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_test
         ,taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_test
         ,taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_test
         ,taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
         ,taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_test
         ,taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_test
         ,taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
         ,taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
         ,taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
         ,taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_test
         ,taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
         ,taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_test
         ,taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_test
         ,taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
         ,taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_test
         ,taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_test
         ,taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_test
         ,taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_test
         ,taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
         ,taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
         ,taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
         ,taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
         ,case
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_COVID_19_EVALUATION_NOTE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_COVID_19_EVALUATION_NOTE_lab_value
         ,case
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value
         ,case
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value
         ,case
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value
         ,case
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value
         ,case
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value
         ,case
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
         ,case
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
         ,case
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
         ,case
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
         ,case
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value) like '%NOT DETECTED%'  then 1
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%DETECTED%'  then 12
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%NOT APPLICABLE%'  then 0
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NORMAL%'  then 4
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%ABNORMAL%'  then 15
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like '%VERY ABNORMAL%'  then 26
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%NEGATIVE%'  then 7
                       when upper(taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value)  like  '%POSITIVE%'  then 18
                       else 0
                       end as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
       
       from {userName}.{algorithmName}_SummaryPatientsCoronavirusLabs cl
       
       ) proc
           
    group by 1,2,3
    
    order by 1,2,3
""")

In [24]:
coronavirusLabsDf.toPandas().to_excel("test2.xlsx")

In [25]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_coronavirusLabs_ml""")

DataFrame[]

In [26]:
coronavirusLabsDf.write.saveAsTable(f"""{userName}.{algorithmName}_coronavirusLabs_ml""")

In [16]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsConditions""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- effectiveDate: string (nullable = true)
 |-- has_ABDOMINAL_PAIN_CLIN: integer (nullable = true)
 |-- has_ACUTE_RESPIRATORY_FAILURE_CLIN: integer (nullable = true)
 |-- has_ADVANCED_ILLNESS_CLIN: integer (nullable = true)
 |-- has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN: integer (nullable = true)
 |-- has_ANEMIA_CLIN: integer (nullable = true)
 |-- has_ANXIETY_CLIN: integer (nullable = true)
 |-- has_AORTIC_STENOSIS_CLIN: integer (nullable = true)
 |-- has_ARRHYTHMIA_CLIN: integer (nullable = true)
 |-- has_ASTHMA_CLIN: integer (nullable = true)
 |-- has_ATHEROSCLEROSIS_CLIN: integer (nullable = true)
 |-- has_ATRIAL_FIBRILLATION_CLIN: integer (nullable = true)
 |-- has_ATRIAL_FLUTTER_CLIN: integer (nullable = true)
 |-- has_BACTERIAL_INFECTION_OTHER_CLIN: integer (nullable = true)
 |-- has_BED_BOUND_CLIN: integer (nullable = true)
 |-- has_BENIGN_PROSTATIC_HYPERP

In [27]:
conditionDf = spark.sql(f"""
select c.tenant
      ,c.personId
      ,c.encounterId
      ,max(has_ABDOMINAL_PAIN_CLIN) as has_ABDOMINAL_PAIN_CLIN
,max(has_ACUTE_RESPIRATORY_FAILURE_CLIN) as has_ACUTE_RESPIRATORY_FAILURE_CLIN
,max(has_ADVANCED_ILLNESS_CLIN) as has_ADVANCED_ILLNESS_CLIN
,max(has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN) as has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN
,max(has_ANEMIA_CLIN) as has_ANEMIA_CLIN
,max(has_ANXIETY_CLIN) as has_ANXIETY_CLIN
,max(has_AORTIC_STENOSIS_CLIN) as has_AORTIC_STENOSIS_CLIN
,max(has_ARRHYTHMIA_CLIN) as has_ARRHYTHMIA_CLIN
,max(has_ASTHMA_CLIN) as has_ASTHMA_CLIN
,max(has_ATHEROSCLEROSIS_CLIN) as has_ATHEROSCLEROSIS_CLIN
,max(has_ATRIAL_FIBRILLATION_CLIN) as has_ATRIAL_FIBRILLATION_CLIN
,max(has_ATRIAL_FLUTTER_CLIN) as has_ATRIAL_FLUTTER_CLIN
,max(has_BACTERIAL_INFECTION_OTHER_CLIN) as has_BACTERIAL_INFECTION_OTHER_CLIN
,max(has_BED_BOUND_CLIN) as has_BED_BOUND_CLIN
,max(has_BENIGN_PROSTATIC_HYPERPLASIA_CLIN) as has_BENIGN_PROSTATIC_HYPERPLASIA_CLIN
,max(has_BIPOLAR_DISORDER_CLIN) as has_BIPOLAR_DISORDER_CLIN
,max(has_BMI_ABOVE_NORMAL_ADULT_CLIN) as has_BMI_ABOVE_NORMAL_ADULT_CLIN
,max(has_BMI_ABOVE_NORMAL_SENIOR_ADULT_CLIN) as has_BMI_ABOVE_NORMAL_SENIOR_ADULT_CLIN
,max(has_BMI_BELOW_NORMAL_ADULT_CLIN) as has_BMI_BELOW_NORMAL_ADULT_CLIN
,max(has_BMI_BELOW_NORMAL_SENIOR_ADULT_CLIN) as has_BMI_BELOW_NORMAL_SENIOR_ADULT_CLIN
,max(has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_30_AND_LESS_THAN_35_CLIN) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_30_AND_LESS_THAN_35_CLIN
,max(has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_35_AND_LESS_THAN_40_CLIN) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_35_AND_LESS_THAN_40_CLIN
,max(has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_40_CLIN) as has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_40_CLIN
,max(has_BRADYARRHYTHMIA_CLIN) as has_BRADYARRHYTHMIA_CLIN
,max(has_CARDIOMYOPATHY_CLIN) as has_CARDIOMYOPATHY_CLIN
,max(has_CARDIOVASCULAR_DEVICES_CLIN) as has_CARDIOVASCULAR_DEVICES_CLIN
,max(has_CEREBROVASCULAR_ACCIDENT_CLIN) as has_CEREBROVASCULAR_ACCIDENT_CLIN
,max(has_CESAREAN_DELIVERY_CLIN) as has_CESAREAN_DELIVERY_CLIN
,max(has_CHEST_PAIN_CLIN) as has_CHEST_PAIN_CLIN
,max(has_CHEST_XRAY_CLIN) as has_CHEST_XRAY_CLIN
,max(has_CHRONIC_AIRWAY_OBSTRUCTION_CLIN) as has_CHRONIC_AIRWAY_OBSTRUCTION_CLIN
,max(has_CHRONIC_KIDNEY_DISEASE_CLIN) as has_CHRONIC_KIDNEY_DISEASE_CLIN
,max(has_CHRONIC_KIDNEY_DISEASE_STAGE_3_CLIN) as has_CHRONIC_KIDNEY_DISEASE_STAGE_3_CLIN
,max(has_CHRONIC_KIDNEY_DISEASE_STAGE_4_CLIN) as has_CHRONIC_KIDNEY_DISEASE_STAGE_4_CLIN
,max(has_CHRONIC_KIDNEY_DISEASE_STAGE_5_CLIN) as has_CHRONIC_KIDNEY_DISEASE_STAGE_5_CLIN
,max(has_CHRONIC_LIVER_DISEASE_AND_CIRRHOSIS_CLIN) as has_CHRONIC_LIVER_DISEASE_AND_CIRRHOSIS_CLIN
,max(has_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_CLIN) as has_CHRONIC_OBSTRUCTIVE_PULMONARY_DISEASE_CLIN
,max(has_CHRONIC_PAIN_CLIN) as has_CHRONIC_PAIN_CLIN
,max(has_CHRONIC_RESPIRATORY_FAILURE_CLIN) as has_CHRONIC_RESPIRATORY_FAILURE_CLIN
,max(has_CKD_STAGE_3_4_OR_5_CLIN) as has_CKD_STAGE_3_4_OR_5_CLIN
,max(has_COGNITIVE_IMPAIRMENT_CLIN) as has_COGNITIVE_IMPAIRMENT_CLIN
,max(has_CONDUCTION_DISORDER_CLIN) as has_CONDUCTION_DISORDER_CLIN
,max(has_CONGESTIVE_HEART_FAILURE_CLIN) as has_CONGESTIVE_HEART_FAILURE_CLIN
,max(has_CONSTIPATION_CLIN) as has_CONSTIPATION_CLIN
,max(has_CORONARY_ARTERY_DISEASE_CLIN) as has_CORONARY_ARTERY_DISEASE_CLIN
,max(has_CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN) as has_CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN
,max(has_CORTICOSTEROID_ADVERSE_REACTION_CLIN) as has_CORTICOSTEROID_ADVERSE_REACTION_CLIN
,max(has_CORTICOSTEROID_LONG_TERM_USE_CLIN) as has_CORTICOSTEROID_LONG_TERM_USE_CLIN
,max(has_COUGH_CLIN) as has_COUGH_CLIN
,max(has_CURRENT_PREGNANCY_NOT_DELIVERED_CLIN) as has_CURRENT_PREGNANCY_NOT_DELIVERED_CLIN
,max(has_DEMENTIA_CLIN) as has_DEMENTIA_CLIN
,max(has_DEVICE_CLIN) as has_DEVICE_CLIN
,max(has_DIABETES_MELLITUS_TYPE_2_CLIN) as has_DIABETES_MELLITUS_TYPE_2_CLIN
,max(has_DIABETIC_KETOACIDOSIS_CLIN) as has_DIABETIC_KETOACIDOSIS_CLIN
,max(has_DIABETIC_NEPHROPATHY_CLIN) as has_DIABETIC_NEPHROPATHY_CLIN
,max(has_DIAPHRAGM_AND_ABDOMINAL_WALL_CLIN) as has_DIAPHRAGM_AND_ABDOMINAL_WALL_CLIN
,max(has_DIARRHEA_CLIN) as has_DIARRHEA_CLIN
,max(has_DIZZINESS_CLIN) as has_DIZZINESS_CLIN
,max(has_DYSRHYTHMIA_CLIN) as has_DYSRHYTHMIA_CLIN
,max(has_ECG_CLIN) as has_ECG_CLIN
,max(has_EMPHYSEMA_CLIN) as has_EMPHYSEMA_CLIN
,max(has_ENCEPHALOPATHY_CLIN) as has_ENCEPHALOPATHY_CLIN
,max(has_ENCEPHALOPATHY_UNSPECIFIED_CLIN) as has_ENCEPHALOPATHY_UNSPECIFIED_CLIN
,max(has_END_STAGE_RENAL_DISEASE_CLIN) as has_END_STAGE_RENAL_DISEASE_CLIN
,max(has_ENDOCARDIUM_DISEASE_CLIN) as has_ENDOCARDIUM_DISEASE_CLIN
,max(has_ENDOCRINE_DISORDER_CLIN) as has_ENDOCRINE_DISORDER_CLIN
,max(has_FALL_CLIN) as has_FALL_CLIN
,max(has_FALL_RISK_CLIN) as has_FALL_RISK_CLIN
,max(has_FAMILY_HISTORY_OF_DIABETES_MELLITUS_CLIN) as has_FAMILY_HISTORY_OF_DIABETES_MELLITUS_CLIN
,max(has_FATIGUE_CLIN) as has_FATIGUE_CLIN
,max(has_FEVER_CLIN) as has_FEVER_CLIN
,max(has_FLUID_OVERLOAD_CLIN) as has_FLUID_OVERLOAD_CLIN
,max(has_GASTROENTERITIS_CLIN) as has_GASTROENTERITIS_CLIN
,max(has_GASTROINTESTINAL_DEVICES_CLIN) as has_GASTROINTESTINAL_DEVICES_CLIN
,max(has_GENERALIZED_ANXIETY_DISORDER_CLIN) as has_GENERALIZED_ANXIETY_DISORDER_CLIN
,max(has_HEADACHE_CLIN) as has_HEADACHE_CLIN
,max(has_HEART_FAILURE_CLIN) as has_HEART_FAILURE_CLIN
,max(has_HEMATOCHEZIA_CLIN) as has_HEMATOCHEZIA_CLIN
,max(has_HOMELESS_CLIN) as has_HOMELESS_CLIN
,max(has_HYPERCHOLESTEROLEMIA_CLIN) as has_HYPERCHOLESTEROLEMIA_CLIN
,max(has_HYPERGLYCEMIA_CLIN) as has_HYPERGLYCEMIA_CLIN
,max(has_HYPERLIPIDEMIA_CLIN) as has_HYPERLIPIDEMIA_CLIN
,max(has_HYPERTENSION_CLIN) as has_HYPERTENSION_CLIN
,max(has_HYPERTENSIVE_CHRONIC_RENAL_DISEASE_CLIN) as has_HYPERTENSIVE_CHRONIC_RENAL_DISEASE_CLIN
,max(has_HYPOGLYCEMIA_CLIN) as has_HYPOGLYCEMIA_CLIN
,max(has_HYPOTENSION_CLIN) as has_HYPOTENSION_CLIN
,max(has_HYSTERECTOMY_NO_CERVIX_CLIN) as has_HYSTERECTOMY_NO_CERVIX_CLIN
,max(has_INSOMNIA_CLIN) as has_INSOMNIA_CLIN
,max(has_IVD_CLIN) as has_IVD_CLIN
,max(has_LIPID_METABOLISM_CLIN) as has_LIPID_METABOLISM_CLIN
,max(has_LIVE_BIRTH_CLIN) as has_LIVE_BIRTH_CLIN
,max(has_LIVER_CIRRHOSIS_CLIN) as has_LIVER_CIRRHOSIS_CLIN
,max(has_LIVER_DISEASE_CLIN) as has_LIVER_DISEASE_CLIN
,max(has_LONG_TERM_OXYGEN_THERAPY_CLIN) as has_LONG_TERM_OXYGEN_THERAPY_CLIN
,max(has_LOW_BACK_PAIN_CLIN) as has_LOW_BACK_PAIN_CLIN
,max(has_MAJOR_DEPRESSIVE_DISORDER_CLIN) as has_MAJOR_DEPRESSIVE_DISORDER_CLIN
,max(has_MORBID_OBESITY_CLIN) as has_MORBID_OBESITY_CLIN
,max(has_MOVEMENT_DISEASE_CLIN) as has_MOVEMENT_DISEASE_CLIN
,max(has_MYOCARDIAL_INFARCTION_CLIN) as has_MYOCARDIAL_INFARCTION_CLIN
,max(has_NAUSEA_CLIN) as has_NAUSEA_CLIN
,max(has_NON_AMBULATORY_CLIN) as has_NON_AMBULATORY_CLIN
,max(has_OBESITY_CLIN) as has_OBESITY_CLIN
,max(has_OBSTRUCTIVE_SLEEP_APNEA_CLIN) as has_OBSTRUCTIVE_SLEEP_APNEA_CLIN
,max(has_OSTEOPOROSIS_CLIN) as has_OSTEOPOROSIS_CLIN
,max(has_OTHER_GASTROINTESTINAL_CLIN) as has_OTHER_GASTROINTESTINAL_CLIN
,max(has_PLEURISY_CLIN) as has_PLEURISY_CLIN
,max(has_PNEUMONIA_CLIN) as has_PNEUMONIA_CLIN
,max(has_PNEUMONIA_NON_VIRAL_CLIN) as has_PNEUMONIA_NON_VIRAL_CLIN
,max(has_PREDIABETES_CLIN) as has_PREDIABETES_CLIN
,max(has_PREGNANCY_CLIN) as has_PREGNANCY_CLIN
,max(has_PSYCHOTIC_DISORDER_CLIN) as has_PSYCHOTIC_DISORDER_CLIN
,max(has_PULMONARY_EMBOLISM_CLIN) as has_PULMONARY_EMBOLISM_CLIN
,max(has_PULMONARY_HYPERTENSION_CLIN) as has_PULMONARY_HYPERTENSION_CLIN
,max(has_RENAL_DIALYSIS_CLIN) as has_RENAL_DIALYSIS_CLIN
,max(has_RENAL_FAILURE_ACUTE_CLIN) as has_RENAL_FAILURE_ACUTE_CLIN
,max(has_RENAL_FAILURE_CHRONIC_CLIN) as has_RENAL_FAILURE_CHRONIC_CLIN
,max(has_RESPIRATORY_DEVICES_CLIN) as has_RESPIRATORY_DEVICES_CLIN
,max(has_RESPIRATORY_DISEASE_SPECIFIED_OTHER_CLIN) as has_RESPIRATORY_DISEASE_SPECIFIED_OTHER_CLIN
,max(has_RESPIRATORY_DISTRESS_CLIN) as has_RESPIRATORY_DISTRESS_CLIN
,max(has_RHABDOMYOLYSIS_CLIN) as has_RHABDOMYOLYSIS_CLIN
,max(has_RHEUMATOID_ARTHRITIS_CLIN) as has_RHEUMATOID_ARTHRITIS_CLIN
,max(has_SCHIZOPHRENIA_CLIN) as has_SCHIZOPHRENIA_CLIN
,max(has_SCHIZOPHRENIA_OR_PSYCHOTIC_DISORDER_CLIN) as has_SCHIZOPHRENIA_OR_PSYCHOTIC_DISORDER_CLIN
,max(has_SEIZURE_CLIN) as has_SEIZURE_CLIN
,max(has_SHORTNESS_OF_BREATH_CLIN) as has_SHORTNESS_OF_BREATH_CLIN
,max(has_SMOKELESS_TOBACCO_USER_CLIN) as has_SMOKELESS_TOBACCO_USER_CLIN
,max(has_SUBSTANCE_ABUSE_CLIN) as has_SUBSTANCE_ABUSE_CLIN
,max(has_SUICIDAL_IDEATION_CLIN) as has_SUICIDAL_IDEATION_CLIN
,max(has_TOBACCO_USER_CLIN) as has_TOBACCO_USER_CLIN
,max(has_UPPER_RESPIRATORY_INFECTION_ACUTE_CLIN) as has_UPPER_RESPIRATORY_INFECTION_ACUTE_CLIN
,max(has_URINARY_INCONTINENCE_CLIN) as has_URINARY_INCONTINENCE_CLIN
,max(has_URINARY_TRACT_INFECTION_CLIN) as has_URINARY_TRACT_INFECTION_CLIN
,max(has_VAGINAL_DELIVERY_CLIN) as has_VAGINAL_DELIVERY_CLIN
,max(has_VERTIGO_CLIN) as has_VERTIGO_CLIN
,max(has_VITAMIN_D_DEFICIENCY_CLIN) as has_VITAMIN_D_DEFICIENCY_CLIN
,max(has_VOMITING_CLIN) as has_VOMITING_CLIN
,max(has_WHEELCHAIR_BOUND_CLIN) as has_WHEELCHAIR_BOUND_CLIN
,max(has_CCS_CATEGORY_101_CORONARY_ATHEROSCLEROSIS_HEART_DISEASE_CLIN) as has_CCS_CATEGORY_101_CORONARY_ATHEROSCLEROSIS_HEART_DISEASE_CLIN
,max(has_CCS_CATEGORY_103_PULMONARY_HEART_DISEASE_CLIN) as has_CCS_CATEGORY_103_PULMONARY_HEART_DISEASE_CLIN
,max(has_CCS_CATEGORY_104_OTHER_AND_ILL_DEFINED_HEART_DISEASE_CLIN) as has_CCS_CATEGORY_104_OTHER_AND_ILL_DEFINED_HEART_DISEASE_CLIN
,max(has_CCS_CATEGORY_105_CONDUCTION_DISORDERS_CLIN) as has_CCS_CATEGORY_105_CONDUCTION_DISORDERS_CLIN
,max(has_CCS_CATEGORY_106_CARDIAC_DYSRHYTHMIAS_CLIN) as has_CCS_CATEGORY_106_CARDIAC_DYSRHYTHMIAS_CLIN
,max(has_CCS_CATEGORY_108_CONGESTIVE_HEART_FAILURE_NONHYPERTENSIVE_CLIN) as has_CCS_CATEGORY_108_CONGESTIVE_HEART_FAILURE_NONHYPERTENSIVE_CLIN
,max(has_CCS_CATEGORY_109_ACUTE_CEREBROVASCULAR_DISEASE_CLIN) as has_CCS_CATEGORY_109_ACUTE_CEREBROVASCULAR_DISEASE_CLIN
,max(has_CCS_CATEGORY_11_CANCER_OF_HEAD_AND_NECK_CLIN) as has_CCS_CATEGORY_11_CANCER_OF_HEAD_AND_NECK_CLIN
,max(has_CCS_CATEGORY_112_TRANSIENT_CEREBRAL_ISCHEMIA_CLIN) as has_CCS_CATEGORY_112_TRANSIENT_CEREBRAL_ISCHEMIA_CLIN
,max(has_CCS_CATEGORY_113_LATE_EFFECTS_OF_CEREBROVASCULAR_DISEASE_CLIN) as has_CCS_CATEGORY_113_LATE_EFFECTS_OF_CEREBROVASCULAR_DISEASE_CLIN
,max(has_CCS_CATEGORY_114_PERIPHERAL_AND_VISCERAL_ATHEROSCLEROSIS_CLIN) as has_CCS_CATEGORY_114_PERIPHERAL_AND_VISCERAL_ATHEROSCLEROSIS_CLIN
,max(has_CCS_CATEGORY_115_AORTIC_PERIPHERAL_VISCERAL_ARTERY_ANEURYSMS_CLIN) as has_CCS_CATEGORY_115_AORTIC_PERIPHERAL_VISCERAL_ARTERY_ANEURYSMS_CLIN
,max(has_CCS_CATEGORY_117_OTHER_CIRCULATORY_DISEASE_CLIN) as has_CCS_CATEGORY_117_OTHER_CIRCULATORY_DISEASE_CLIN
,max(has_CCS_CATEGORY_12_CANCER_OF_ESOPHAGUS_CLIN) as has_CCS_CATEGORY_12_CANCER_OF_ESOPHAGUS_CLIN
,max(has_CCS_CATEGORY_121_OTHER_DISEASES_VEINS_LYMPHATICS_CLIN) as has_CCS_CATEGORY_121_OTHER_DISEASES_VEINS_LYMPHATICS_CLIN
,max(has_CCS_CATEGORY_122_PNEUMONIA_EXCEPT_TB_OR_STD_CLIN) as has_CCS_CATEGORY_122_PNEUMONIA_EXCEPT_TB_OR_STD_CLIN
,max(has_CCS_CATEGORY_127_COPD_BRONCHIECTASIS_CLIN) as has_CCS_CATEGORY_127_COPD_BRONCHIECTASIS_CLIN
,max(has_CCS_CATEGORY_128_ASTHMA_CLIN) as has_CCS_CATEGORY_128_ASTHMA_CLIN
,max(has_CCS_CATEGORY_129_ASPIRATION_PNEUMONITIS_FOOD_VOMITUS_CLIN) as has_CCS_CATEGORY_129_ASPIRATION_PNEUMONITIS_FOOD_VOMITUS_CLIN
,max(has_CCS_CATEGORY_13_CANCER_OF_STOMACH_CLIN) as has_CCS_CATEGORY_13_CANCER_OF_STOMACH_CLIN
,max(has_CCS_CATEGORY_139_GASTRODUODENAL_ULCER_EXCEPT_HEMORRHAGE_CLIN) as has_CCS_CATEGORY_139_GASTRODUODENAL_ULCER_EXCEPT_HEMORRHAGE_CLIN
,max(has_CCS_CATEGORY_14_CANCER_OF_COLON_CLIN) as has_CCS_CATEGORY_14_CANCER_OF_COLON_CLIN
,max(has_CCS_CATEGORY_15_CANCER_OF_RECTUM_AND_ANUS_CLIN) as has_CCS_CATEGORY_15_CANCER_OF_RECTUM_AND_ANUS_CLIN
,max(has_CCS_CATEGORY_151_OTHER_LIVER_DISEASES_CLIN) as has_CCS_CATEGORY_151_OTHER_LIVER_DISEASES_CLIN
,max(has_CCS_CATEGORY_156_NEPHRITIS_NEPHROSIS_RENAL_SCLEROSIS_CLIN) as has_CCS_CATEGORY_156_NEPHRITIS_NEPHROSIS_RENAL_SCLEROSIS_CLIN
,max(has_CCS_CATEGORY_157_ACUTE_UNSPECIFIED_RENAL_FAILURE_CLIN) as has_CCS_CATEGORY_157_ACUTE_UNSPECIFIED_RENAL_FAILURE_CLIN
,max(has_CCS_CATEGORY_158_CHRONIC_KIDNEY_DISEASE_CLIN) as has_CCS_CATEGORY_158_CHRONIC_KIDNEY_DISEASE_CLIN
,max(has_CCS_CATEGORY_16_CANCER_OF_LIVER_AND_INTRAHEPATIC_BILE_DUCT_CLIN) as has_CCS_CATEGORY_16_CANCER_OF_LIVER_AND_INTRAHEPATIC_BILE_DUCT_CLIN
,max(has_CCS_CATEGORY_17_CANCER_OF_PANCREAS_CLIN) as has_CCS_CATEGORY_17_CANCER_OF_PANCREAS_CLIN
,max(has_CCS_CATEGORY_18_CANCER_OF_OTHER_GI_ORGANS_PERITONEUM_CLIN) as has_CCS_CATEGORY_18_CANCER_OF_OTHER_GI_ORGANS_PERITONEUM_CLIN
,max(has_CCS_CATEGORY_19_CANCER_OF_BRONCHUS_LUNG_CLIN) as has_CCS_CATEGORY_19_CANCER_OF_BRONCHUS_LUNG_CLIN
,max(has_CCS_CATEGORY_2_SEPTICEMIA_EXECPT_IN_LABOR_CLIN) as has_CCS_CATEGORY_2_SEPTICEMIA_EXECPT_IN_LABOR_CLIN
,max(has_CCS_CATEGORY_20_CANCER_OTHER_RESPIRATORY_AND_INTRATHORACIC_CLIN) as has_CCS_CATEGORY_20_CANCER_OTHER_RESPIRATORY_AND_INTRATHORACIC_CLIN
,max(has_CCS_CATEGORY_207_PATHOLOGICAL_FRACTURE_CLIN) as has_CCS_CATEGORY_207_PATHOLOGICAL_FRACTURE_CLIN
,max(has_CCS_CATEGORY_21_CANCER_OF_BONE_AND_CONNECTIVE_TISSUE_CLIN) as has_CCS_CATEGORY_21_CANCER_OF_BONE_AND_CONNECTIVE_TISSUE_CLIN
,max(has_CCS_CATEGORY_210_LUPUS_AND_CONNECTIVE_TISSUE_DISORDERS_CLIN) as has_CCS_CATEGORY_210_LUPUS_AND_CONNECTIVE_TISSUE_DISORDERS_CLIN
,max(has_CCS_CATEGORY_211_OTHER_CONNECTIVE_TISSUE_DISEASE_CLIN) as has_CCS_CATEGORY_211_OTHER_CONNECTIVE_TISSUE_DISEASE_CLIN
,max(has_CCS_CATEGORY_213_CARDIAC_CIRCULATORY_CONGENITAL_ANOMALIES_CLIN) as has_CCS_CATEGORY_213_CARDIAC_CIRCULATORY_CONGENITAL_ANOMALIES_CLIN
,max(has_CCS_CATEGORY_22_MELANOMAS_OF_SKIN_CLIN) as has_CCS_CATEGORY_22_MELANOMAS_OF_SKIN_CLIN
,max(has_CCS_CATEGORY_226_FRACTURE_NECK_OF_FEMUR_HIP_CLIN) as has_CCS_CATEGORY_226_FRACTURE_NECK_OF_FEMUR_HIP_CLIN
,max(has_CCS_CATEGORY_23_OTHER_NON_EPITHELIAL_CANCER_OF_SKIN_CLIN) as has_CCS_CATEGORY_23_OTHER_NON_EPITHELIAL_CANCER_OF_SKIN_CLIN
,max(has_CCS_CATEGORY_230_FRACTURE_OF_LOWER_LIMB_CLIN) as has_CCS_CATEGORY_230_FRACTURE_OF_LOWER_LIMB_CLIN
,max(has_CCS_CATEGORY_237_DEVICE_COMPLICATION_IMPLANT_OR_GRAFT_CLIN) as has_CCS_CATEGORY_237_DEVICE_COMPLICATION_IMPLANT_OR_GRAFT_CLIN
,max(has_CCS_CATEGORY_238_COMPLICATIONS_SURGICAL_PROC_OR_MEDICAL_CARE_CLIN) as has_CCS_CATEGORY_238_COMPLICATIONS_SURGICAL_PROC_OR_MEDICAL_CARE_CLIN
,max(has_CCS_CATEGORY_24_CANCER_OF_BREAST_CLIN) as has_CCS_CATEGORY_24_CANCER_OF_BREAST_CLIN
,max(has_CCS_CATEGORY_242_POISONING_BY_OTHER_MEDICATIONS_AND_DRUGS_CLIN) as has_CCS_CATEGORY_242_POISONING_BY_OTHER_MEDICATIONS_AND_DRUGS_CLIN
,max(has_CCS_CATEGORY_244_OTHER_INJURIES_AND_CONDITIONS_DUE_TO_EXTERNAL_CAUSES_CLIN) as has_CCS_CATEGORY_244_OTHER_INJURIES_AND_CONDITIONS_DUE_TO_EXTERNAL_CAUSES_CLIN
,max(has_CCS_CATEGORY_245_SYNCOPE_CLIN) as has_CCS_CATEGORY_245_SYNCOPE_CLIN
,max(has_CCS_CATEGORY_25_CANCER_OF_UTERUS_CLIN) as has_CCS_CATEGORY_25_CANCER_OF_UTERUS_CLIN
,max(has_CCS_CATEGORY_253_ALLERGIC_REACTIONS_CLIN) as has_CCS_CATEGORY_253_ALLERGIC_REACTIONS_CLIN
,max(has_CCS_CATEGORY_258_OTHER_SCREENING_FOR_SUSPECTED_CONDITIONS_CLIN) as has_CCS_CATEGORY_258_OTHER_SCREENING_FOR_SUSPECTED_CONDITIONS_CLIN
,max(has_CCS_CATEGORY_259_RESIDUAL_CODES_UNCLASSIFIED_CLIN) as has_CCS_CATEGORY_259_RESIDUAL_CODES_UNCLASSIFIED_CLIN
,max(has_CCS_CATEGORY_26_CANCER_OF_CERVIX_CLIN) as has_CCS_CATEGORY_26_CANCER_OF_CERVIX_CLIN
,max(has_CCS_CATEGORY_27_CANCER_OF_OVARY_CLIN) as has_CCS_CATEGORY_27_CANCER_OF_OVARY_CLIN
,max(has_CCS_CATEGORY_28_CANCER_OTHER_FEMALE_GENITAL_ORGANS_CLIN) as has_CCS_CATEGORY_28_CANCER_OTHER_FEMALE_GENITAL_ORGANS_CLIN
,max(has_CCS_CATEGORY_29_CANCER_OF_PROSTATE_CLIN) as has_CCS_CATEGORY_29_CANCER_OF_PROSTATE_CLIN
,max(has_CCS_CATEGORY_30_CANCER_OF_TESTIS_CLIN) as has_CCS_CATEGORY_30_CANCER_OF_TESTIS_CLIN
,max(has_CCS_CATEGORY_31_CANCER_OTHER_MALE_GENITAL_ORGANS_CLIN) as has_CCS_CATEGORY_31_CANCER_OTHER_MALE_GENITAL_ORGANS_CLIN
,max(has_CCS_CATEGORY_32_CANCER_OF_BLADDER_CLIN) as has_CCS_CATEGORY_32_CANCER_OF_BLADDER_CLIN
,max(has_CCS_CATEGORY_33_CANCER_KIDNEY_AND_RENAL_PELVIS_CLIN) as has_CCS_CATEGORY_33_CANCER_KIDNEY_AND_RENAL_PELVIS_CLIN
,max(has_CCS_CATEGORY_34_CANCER_OTHER_URINARY_ORGANS_CLIN) as has_CCS_CATEGORY_34_CANCER_OTHER_URINARY_ORGANS_CLIN
,max(has_CCS_CATEGORY_35_CANCER_BRAIN_NERVOUS_SYSTEM_CLIN) as has_CCS_CATEGORY_35_CANCER_BRAIN_NERVOUS_SYSTEM_CLIN
,max(has_CCS_CATEGORY_36_CANCER_OF_THYROID_CLIN) as has_CCS_CATEGORY_36_CANCER_OF_THYROID_CLIN
,max(has_CCS_CATEGORY_37_HODGKINS_DISEASE_CLIN) as has_CCS_CATEGORY_37_HODGKINS_DISEASE_CLIN
,max(has_CCS_CATEGORY_38_NON_HODGKINS_LYMPHOMA_CLIN) as has_CCS_CATEGORY_38_NON_HODGKINS_LYMPHOMA_CLIN
,max(has_CCS_CATEGORY_39_LEUKEMIAS_CLIN) as has_CCS_CATEGORY_39_LEUKEMIAS_CLIN
,max(has_CCS_CATEGORY_40_MULTIPLE_MYELOMA_CLIN) as has_CCS_CATEGORY_40_MULTIPLE_MYELOMA_CLIN
,max(has_CCS_CATEGORY_41_CANCER_OTHER_AND_UNSPECIFIED_PRIMARY_CLIN) as has_CCS_CATEGORY_41_CANCER_OTHER_AND_UNSPECIFIED_PRIMARY_CLIN
,max(has_CCS_CATEGORY_42_SECONDARY_MALIGNANCIES_CLIN) as has_CCS_CATEGORY_42_SECONDARY_MALIGNANCIES_CLIN
,max(has_CCS_CATEGORY_43_MALIGNANT_NEOPLASM_WITHOUT_SPECIFICATION_OF_SITE_CLIN) as has_CCS_CATEGORY_43_MALIGNANT_NEOPLASM_WITHOUT_SPECIFICATION_OF_SITE_CLIN
,max(has_CCS_CATEGORY_49_DIABETES_MELLITUS_WITHOUT_COMPLICATION_CLIN) as has_CCS_CATEGORY_49_DIABETES_MELLITUS_WITHOUT_COMPLICATION_CLIN
,max(has_CCS_CATEGORY_5_HIV_INFECTION_CLIN) as has_CCS_CATEGORY_5_HIV_INFECTION_CLIN
,max(has_CCS_CATEGORY_50_DIABETES_MELLITUS_WITH_COMPLICATIONS_CLIN) as has_CCS_CATEGORY_50_DIABETES_MELLITUS_WITH_COMPLICATIONS_CLIN
,max(has_CCS_CATEGORY_52_NUTRITIONAL_DEFICIENCIES_CLIN) as has_CCS_CATEGORY_52_NUTRITIONAL_DEFICIENCIES_CLIN
,max(has_CCS_CATEGORY_55_FLUID_AND_ELECTROLYTE_DISORDERS_CLIN) as has_CCS_CATEGORY_55_FLUID_AND_ELECTROLYTE_DISORDERS_CLIN
,max(has_CCS_CATEGORY_58_NUTRITIONAL_ENDOCRINE_METABOLIC_DISORDERS_CLIN) as has_CCS_CATEGORY_58_NUTRITIONAL_ENDOCRINE_METABOLIC_DISORDERS_CLIN
,max(has_CCS_CATEGORY_6_HEPATITIS_CLIN) as has_CCS_CATEGORY_6_HEPATITIS_CLIN
,max(has_CCS_CATEGORY_62_COAGULATION_HEMORRHAGIC_DISORDERS_CLIN) as has_CCS_CATEGORY_62_COAGULATION_HEMORRHAGIC_DISORDERS_CLIN
,max(has_CCS_CATEGORY_64_OTHER_HEMATOLOGIC_CONDITIONS_CLIN) as has_CCS_CATEGORY_64_OTHER_HEMATOLOGIC_CONDITIONS_CLIN
,max(has_CCS_CATEGORY_659_SCHIZOPHRENIA_AND_OTHER_PSYCHOTIC_DISORDERS_CLIN) as has_CCS_CATEGORY_659_SCHIZOPHRENIA_AND_OTHER_PSYCHOTIC_DISORDERS_CLIN
,max(has_CCS_CATEGORY_660_ALCOHOL_RELATED_DISORDERS_CLIN) as has_CCS_CATEGORY_660_ALCOHOL_RELATED_DISORDERS_CLIN
,max(has_CCS_CATEGORY_663_SCREEN_AND_HISTORY_MENTAL_HEALTH_AND_SUB_ABUSE_CODES_CLIN) as has_CCS_CATEGORY_663_SCREEN_AND_HISTORY_MENTAL_HEALTH_AND_SUB_ABUSE_CODES_CLIN
,max(has_CCS_CATEGORY_7_VIRAL_INFECTION_CLIN) as has_CCS_CATEGORY_7_VIRAL_INFECTION_CLIN
,max(has_CCS_CATEGORY_77_ENCEPHALITIS_CLIN) as has_CCS_CATEGORY_77_ENCEPHALITIS_CLIN
,max(has_CCS_CATEGORY_81_HEREDITARY_DEGENERATIVE_NERVOUS_SYSTEM_CONDITIONS_CLIN) as has_CCS_CATEGORY_81_HEREDITARY_DEGENERATIVE_NERVOUS_SYSTEM_CONDITIONS_CLIN
,max(has_CCS_CATEGORY_95_OTHER_NERVOUS_SYSTEM_DISORDERS_CLIN) as has_CCS_CATEGORY_95_OTHER_NERVOUS_SYSTEM_DISORDERS_CLIN
,max(has_CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN) as has_CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN
,max(has_CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN) as has_CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN
,max(has_STILLBIRTH_CLIN) as has_STILLBIRTH_CLIN
,max(has_TERMINATION_OF_PREGNANCY_CLIN) as has_TERMINATION_OF_PREGNANCY_CLIN
,max(has_HOSPICE_CARE_CLIN) as has_HOSPICE_CARE_CLIN
,max(has_SEPSIS_CLIN) as has_SEPSIS_CLIN
,max(has_COMPLICATIONS_OF_PREGNANCY_CHILDBIRTH_AND_THE_PUERPERIUM_CLIN) as has_COMPLICATIONS_OF_PREGNANCY_CHILDBIRTH_AND_THE_PUERPERIUM_CLIN
,MAX(has_CANCER_CLIN) as has_CANCER_CLIN
,MAX(has_DOWN_SYNDROME_TRISOMY_21_CLIN) as has_DOWN_SYNDROME_TRISOMY_21_CLIN
,MAX(has_HEART_TRANSPLANT_CLIN) as has_HEART_TRANSPLANT_CLIN
,MAX(has_LIVER_TRANSPLANT_CLIN) as has_LIVER_TRANSPLANT_CLIN
,MAX(has_LUNG_TRANSPLANT_CLIN) as has_LUNG_TRANSPLANT_CLIN
,MAX(has_RENAL_TRANSPLANT_CLIN) as has_RENAL_TRANSPLANT_CLIN

  from {userName}.{algorithmName}_SummaryPatientsConditions c
  
  group by 1,2,3
  order by 1,2,3

""")

In [28]:
conditionDf.toPandas().to_excel("test3.xlsx")

In [29]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_conditions_ml""")

DataFrame[]

In [30]:
conditionDf.write.saveAsTable(f"""{userName}.{algorithmName}_conditions_ml""")

In [31]:
spark.sql(f"""select * from {userName}.{algorithmName}_conditions_ml""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- has_ABDOMINAL_PAIN_CLIN: integer (nullable = true)
 |-- has_ACUTE_RESPIRATORY_FAILURE_CLIN: integer (nullable = true)
 |-- has_ADVANCED_ILLNESS_CLIN: integer (nullable = true)
 |-- has_ALCOHOL_ABUSE_OR_DEPENDENCE_CLIN: integer (nullable = true)
 |-- has_ANEMIA_CLIN: integer (nullable = true)
 |-- has_ANXIETY_CLIN: integer (nullable = true)
 |-- has_AORTIC_STENOSIS_CLIN: integer (nullable = true)
 |-- has_ARRHYTHMIA_CLIN: integer (nullable = true)
 |-- has_ASTHMA_CLIN: integer (nullable = true)
 |-- has_ATHEROSCLEROSIS_CLIN: integer (nullable = true)
 |-- has_ATRIAL_FIBRILLATION_CLIN: integer (nullable = true)
 |-- has_ATRIAL_FLUTTER_CLIN: integer (nullable = true)
 |-- has_BACTERIAL_INFECTION_OTHER_CLIN: integer (nullable = true)
 |-- has_BED_BOUND_CLIN: integer (nullable = true)
 |-- has_BENIGN_PROSTATIC_HYPERPLASIA_CLIN: integer (nullable = true)
 |-- ha

In [49]:
numberConditionsDf = spark.sql(f"""
select tenant
      ,personId
      ,encounterId
      ,if((hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) = 1, 1, 0 ) as hasOneCondition
      ,if((hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) >= 2
          and (hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) <= 5, 1, 0 ) as hasFromTwoToFiveConditions  
      ,if((hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) >= 6
          and (hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) <= 10, 1, 0 ) as hasFromSixToTenConditions 
       ,if((hasAsthmaCondition + hasCancerCondition + hasDiabetesCondition + hasDownSyndromeCondition
           + isDugAbuser + hasHeartCondition + hasHIVCondition + hasHypertensionCondition
           + hasKidneyCondition + hasLiverCondition + hasLungCondition + hasObesityCondition
           + isPregnant + isTobaccoUser + hasStrokeCondition + hasTransplant) > 10, 1, 0 ) as hasElevenOrMoreConditions    
from (           
    select tenant
          ,personId
          ,encounterId
          ,if(has_CCS_CATEGORY_128_ASTHMA_CLIN = 1, 1, 0) as hasAsthmaCondition
          ,if(has_CANCER_CLIN = 1, 1, 0) as hasCancerCondition
          ,if(has_CCS_CATEGORY_49_DIABETES_MELLITUS_WITHOUT_COMPLICATION_CLIN = 1
               or has_CCS_CATEGORY_50_DIABETES_MELLITUS_WITH_COMPLICATIONS_CLIN = 1, 1, 0) as hasDiabetesCondition
          ,if(has_DOWN_SYNDROME_TRISOMY_21_CLIN = 1, 1, 0) as hasDownSyndromeCondition
          ,if(has_DOWN_SYNDROME_TRISOMY_21_CLIN = 1, 1, 0) as isDugAbuser
          ,if(has_CCS_CATEGORY_101_CORONARY_ATHEROSCLEROSIS_HEART_DISEASE_CLIN = 1
               or has_CCS_CATEGORY_103_PULMONARY_HEART_DISEASE_CLIN = 1
               or has_CCS_CATEGORY_104_OTHER_AND_ILL_DEFINED_HEART_DISEASE_CLIN = 1
               or has_CCS_CATEGORY_105_CONDUCTION_DISORDERS_CLIN = 1
               or has_CCS_CATEGORY_106_CARDIAC_DYSRHYTHMIAS_CLIN = 1
               or has_CCS_CATEGORY_108_CONGESTIVE_HEART_FAILURE_NONHYPERTENSIVE_CLIN = 1
               or has_CCS_CATEGORY_97_PERI_ENDO_MYOCARDITIS_CARDIOMYOPATHY_CLIN = 1, 1, 0) as hasHeartCondition
          ,if(has_CCS_CATEGORY_5_HIV_INFECTION_CLIN = 1, 1, 0) as hasHIVCondition
          ,if(has_CCS_CATEGORY_99_HYPERTENSION_AND_SECONDARY_HYPERTENSION_CLIN = 1, 1, 0) as hasHypertensionCondition
          ,if(has_CCS_CATEGORY_156_NEPHRITIS_NEPHROSIS_RENAL_SCLEROSIS_CLIN = 1
               or has_CCS_CATEGORY_157_ACUTE_UNSPECIFIED_RENAL_FAILURE_CLIN = 1
               or has_CCS_CATEGORY_158_CHRONIC_KIDNEY_DISEASE_CLIN = 1, 1, 0) as hasKidneyCondition
          ,if(has_CCS_CATEGORY_151_OTHER_LIVER_DISEASES_CLIN = 1, 1, 0) as hasLiverCondition
          ,if(has_CCS_CATEGORY_127_COPD_BRONCHIECTASIS_CLIN = 1, 1, 0) as hasLungCondition
          ,if(has_MORBID_OBESITY_CLIN = 1
               or has_OBESITY_CLIN = 1
               or has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_30_AND_LESS_THAN_35_CLIN = 1 
               or has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_35_AND_LESS_THAN_40_CLIN = 1 
               or has_BODY_MASS_INDEX_BMI_GREATER_THAN_OR_EQUAL_TO_40_CLIN = 1, 1, 0) as hasObesityCondition
          ,if(has_CURRENT_PREGNANCY_NOT_DELIVERED_CLIN = 1
               or has_PREGNANCY_CLIN = 1, 1, 0) as isPregnant
          ,if(has_SMOKELESS_TOBACCO_USER_CLIN = 1
               or has_TOBACCO_USER_CLIN = 1, 1, 0) as isTobaccoUser
          ,if(has_CCS_CATEGORY_109_ACUTE_CEREBROVASCULAR_DISEASE_CLIN = 1
               or has_CCS_CATEGORY_112_TRANSIENT_CEREBRAL_ISCHEMIA_CLIN = 1
               or has_CCS_CATEGORY_113_LATE_EFFECTS_OF_CEREBROVASCULAR_DISEASE_CLIN = 1, 1, 0) as hasStrokeCondition
          ,if(has_HEART_TRANSPLANT_CLIN = 1
               or has_LIVER_TRANSPLANT_CLIN = 1
               or has_LUNG_TRANSPLANT_CLIN = 1
               or has_RENAL_TRANSPLANT_CLIN = 1, 1, 0) as hasTransplant

           from {userName}.{algorithmName}_conditions_ml ) 
      
   order by 1,2,3   

""") 

In [46]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_numberConditions_ml""")

DataFrame[]

In [50]:
numberConditionsDf.write.saveAsTable(f"""{userName}.{algorithmName}_numberConditions_ml""")

In [51]:
numberConditionsDf.limit(25).toPandas()

,tenant,personId,encounterId,hasOneCondition,hasFromTwoToFiveConditions,hasFromSixToTenConditions,hasElevenOrMoreConditions
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,1,0,0,0
1,1,002756bd-6290-4982-9c69-f879158503ab,32d82cb7-69b8-4d9d-ab2d-b97e14c59927,0,1,0,0
2,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,0,0,0,0
3,1,004f0a70-f538-4e7a-b426-3b7306a59a80,ee12825a-d8a3-4e64-8de6-cd5228c8c16f,0,1,0,0
4,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,0,1,0,0
5,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,6a06b5e8-cdd5-4e98-84f4-1238237dcc8e,0,1,0,0
6,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,aa061342-b172-470d-84b0-f9fe41f63beb,0,1,0,0
7,1,00a50ac0-9c00-43ba-9d25-0dcc0d23549f,757500da-0380-496f-9712-32eebe6004d7,1,0,0,0
8,1,00a8ceb1-b0bc-40ef-9de7-6e88dc900274,ac4062df-18a5-44b1-b307-ec4bfb5908a9,1,0,0,0
9,1,00c9804e-438c-4a7c-95b3-c17718e5b4cf,29a1e91d-468f-46a9-b769-6c03b15f3c25,0,0,1,0


In [31]:
# spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsConditions
#    order by 1, 2, 3""").limit(5).toPandas().to_excel("test3.xlsx")

In [23]:
spark.sql(f"""select * from {userName}.{algorithmName}_HospitalProcedure""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- procedureId: string (nullable = true)
 |-- on_mechanical_ventilation: integer (nullable = true)
 |-- had_chest_ct_scan: integer (nullable = true)
 |-- had_chest_mri: integer (nullable = true)
 |-- had_chest_ultrasound: integer (nullable = true)
 |-- had_chest_xray: integer (nullable = true)
 |-- serviceStartDate: string (nullable = true)
 |-- serviceEndDate: string (nullable = true)



In [55]:
procedureDf = spark.sql(f"""
select p.tenant
      ,p.personId
      ,p.encounterId
      ,max(on_mechanical_ventilation) as on_mechanical_ventilation
,max(had_chest_ct_scan) as had_chest_ct_scan
,max(had_chest_mri) as had_chest_mri
,max(had_chest_ultrasound) as had_chest_ultrasound
,max(had_chest_xray) as had_chest_xray
   from {userName}.{algorithmName}_HospitalProcedure p
   
   group by 1,2,3
   order by 1,2,3
""")

In [56]:
procedureDf.limit(25).toPandas()

,tenant,personId,encounterId,on_mechanical_ventilation,had_chest_ct_scan,had_chest_mri,had_chest_ultrasound,had_chest_xray
0,1,01b2976d-d404-4b8d-966e-cfe99c6e3c09,b8c5dfc4-cd68-4091-bbeb-06ff7c7eda44,1,0,0,0,0
1,1,024f8035-10e5-4bb2-9a39-c4fda12eb729,6a675b01-86af-40eb-abb1-1e375cb4906b,1,0,0,0,0
2,1,02565cd3-990f-48a9-a1c8-03100392fb38,df11cb43-006e-4d0a-98d9-86b91a238580,1,0,0,0,0
3,1,027acb52-785a-4c84-b6f7-4508e4a52531,ac70b3b7-61d5-4732-a8fe-3bd33215a3f1,1,0,0,0,0
4,1,07c5a862-7680-41cf-85a3-e4a861e90790,2a6b1552-d09f-41d5-9559-c2afd7e0bf12,1,0,0,0,0
5,1,08bc997c-f4d9-4fc1-b80d-4a7df46c9218,cb2e19fd-4b8d-49ea-a46f-ead39ae31bae,0,0,0,0,1
6,1,0948b0d8-f141-4fde-97e8-98578d2e8db2,23eaa340-f372-4789-92dd-f41df8efe5e1,1,0,0,0,0
7,1,10096837-85d2-4b66-bf8a-56e054a6a7e4,101e59b4-7038-4794-83d4-8e8b7114111e,1,0,0,0,0
8,1,14ac1ff5-7a22-469b-87b5-98884e399fd7,c90ad3d2-375e-44fb-9915-c4e29aa553d3,1,0,0,0,0
9,1,15a996b4-cac0-43c2-ac66-6d623f437dc5,6f85a670-62ec-4d73-a602-cd7579d043b8,1,0,0,0,0


In [52]:
spark.sql(f"""select * from {userName}.{algorithmName}_HospitalProcedure
   order by 1, 2, 3""").limit(5).toPandas()

,tenant,personId,encounterId,procedureId,on_mechanical_ventilation,had_chest_ct_scan,had_chest_mri,had_chest_ultrasound,had_chest_xray,serviceStartDate,serviceEndDate
0,1,01b2976d-d404-4b8d-966e-cfe99c6e3c09,b8c5dfc4-cd68-4091-bbeb-06ff7c7eda44,1617ac27-e07e-44b2-9f36-9f0cad4b953f,1,0,0,0,0,2021-01-19T07:00:00+00:00,
1,1,01b2976d-d404-4b8d-966e-cfe99c6e3c09,b8c5dfc4-cd68-4091-bbeb-06ff7c7eda44,cfd9d5f3-78d8-49dc-b8be-6c1f647b82c6,1,0,0,0,0,2021-01-08T07:00:00+00:00,
2,1,024f8035-10e5-4bb2-9a39-c4fda12eb729,6a675b01-86af-40eb-abb1-1e375cb4906b,e6b0fefe-b53d-4d86-9ec5-10395d56e615,1,0,0,0,0,2020-08-21T06:00:00+00:00,
3,1,02565cd3-990f-48a9-a1c8-03100392fb38,df11cb43-006e-4d0a-98d9-86b91a238580,4385ecec-c59d-4b94-9526-1a5ff437be87,1,0,0,0,0,2020-05-12T06:00:00+00:00,
4,1,027acb52-785a-4c84-b6f7-4508e4a52531,ac70b3b7-61d5-4732-a8fe-3bd33215a3f1,fa353d59-c7a1-4f42-8fd0-007b667524ca,1,0,0,0,0,2020-11-15T07:00:00+00:00,


In [58]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_procedures_ml""")

DataFrame[]

In [59]:
procedureDf.write.saveAsTable(f"""{userName}.{algorithmName}_procedures_ml""")

In [60]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummarydPatientsLabs""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_id: string (nullable = true)
 |-- taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id: string (nullable = true)
 |-- taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id: string (nullable = true)
 |-- taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id: string (nullable = true)
 |-- taken_PMV_BLOOD_AUTO_lab_id: string (nullable = true)
 |-- taken_HCT_BLOOD_AUTO_lab_id: string (nullable = true)
 |-- taken_RDW_RATIO_AUTO_lab_id: string (nullable = true)
 |-- taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_id: string (nullable = true)
 |-- taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_id: string (nullable = true)
 |-- taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_id: string (nullable = true)
 |-- taken_MO

In [9]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummarydPatientsLabs
   order by 1, 2, 3""").limit(500).toPandas().to_excel("test4.xlsx")

In [16]:
labDf = spark.sql(f"""
select tenant
      ,personId
      ,encounterId
,max(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_test) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_test
,max(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
,max(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
,max(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
,max(taken_PMV_BLOOD_AUTO_lab_test) as taken_PMV_BLOOD_AUTO_lab_test
,max(taken_HCT_BLOOD_AUTO_lab_test) as taken_HCT_BLOOD_AUTO_lab_test
,max(taken_RDW_RATIO_AUTO_lab_test) as taken_RDW_RATIO_AUTO_lab_test
,max(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_test) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_test
,max(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_test) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_test
,max(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_test) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_test
,max(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_test) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_test
,max(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_test) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_test
,max(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_test) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_test
,max(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
,max(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_test) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
,max(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_test) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_test
,max(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_test) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
,max(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_test) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_test
,max(taken_MCV_AUTO_lab_test) as taken_MCV_AUTO_lab_test
,max(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_test) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_test
,max(taken_MCH_AUTO_lab_test) as taken_MCH_AUTO_lab_test
,max(taken_HGB_MASSVOLUME_BLOOD_lab_test) as taken_HGB_MASSVOLUME_BLOOD_lab_test
,max(taken_PROTHROMBIN_TIME_lab_test) as taken_PROTHROMBIN_TIME_lab_test
,max(taken_COLOR_URINE_lab_test) as taken_COLOR_URINE_lab_test
,max(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation
,max(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_PMV_BLOOD_AUTO_lab_interpretation) as taken_PMV_BLOOD_AUTO_lab_interpretation
,max(taken_HCT_BLOOD_AUTO_lab_interpretation) as taken_HCT_BLOOD_AUTO_lab_interpretation
,max(taken_RDW_RATIO_AUTO_lab_interpretation) as taken_RDW_RATIO_AUTO_lab_interpretation
,max(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation
,max(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation
,max(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation
,max(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation
,max(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation
,max(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
,max(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
,max(taken_MCV_AUTO_lab_interpretation) as taken_MCV_AUTO_lab_interpretation
,max(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation
,max(taken_MCH_AUTO_lab_interpretation) as taken_MCH_AUTO_lab_interpretation
,max(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation) as taken_HGB_MASSVOLUME_BLOOD_lab_interpretation
,max(taken_PROTHROMBIN_TIME_lab_interpretation) as taken_PROTHROMBIN_TIME_lab_interpretation
,max(taken_COLOR_URINE_lab_interpretation) as taken_COLOR_URINE_lab_interpretation
      
      

from (
    select l.tenant
          ,l.personId
          ,l.encounterId
          ,l.serviceDate
       ,taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_test
    ,taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
    ,taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
    ,taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
    ,taken_PMV_BLOOD_AUTO_lab_test
    ,taken_HCT_BLOOD_AUTO_lab_test
    ,taken_RDW_RATIO_AUTO_lab_test
    ,taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_test
    ,taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_test
    ,taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_test
    ,taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_test
    ,taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_test
    ,taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_test
    ,taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
    ,taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
    ,taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_test
    ,taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
    ,taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_test
    ,taken_MCV_AUTO_lab_test
    ,taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_test
    ,taken_MCH_AUTO_lab_test
    ,taken_HGB_MASSVOLUME_BLOOD_lab_test
    ,taken_PROTHROMBIN_TIME_lab_test
    ,taken_COLOR_URINE_lab_test

     ,case
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation 
     ,case
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation 
     ,case
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation 
     ,case
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation 
     ,case
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_PMV_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_PMV_BLOOD_AUTO_lab_interpretation 
     ,case
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_HCT_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_HCT_BLOOD_AUTO_lab_interpretation 
     ,case
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_RDW_RATIO_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_RDW_RATIO_AUTO_lab_interpretation 
     ,case
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation 
     ,case
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation 
     ,case
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation 
     ,case
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation 
     ,case
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation 
      ,case
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation 
     ,case
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,case
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,case
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
      ,case
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
      ,case
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,case
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
      ,case
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation 
      ,case
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation 
      ,case
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation 
      ,case
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation 
      ,case
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
     ,case
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,case
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,case
                   when upper(taken_MCV_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MCV_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MCV_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MCV_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MCV_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MCV_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MCV_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MCV_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MCV_AUTO_lab_interpretation
     ,case
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation
     ,case
                   when upper(taken_MCH_AUTO_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_MCH_AUTO_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_MCH_AUTO_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_MCH_AUTO_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_MCH_AUTO_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_MCH_AUTO_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_MCH_AUTO_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_MCH_AUTO_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_MCH_AUTO_lab_interpretation
     ,case
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_HGB_MASSVOLUME_BLOOD_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_HGB_MASSVOLUME_BLOOD_lab_interpretation
     ,case
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_PROTHROMBIN_TIME_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_PROTHROMBIN_TIME_lab_interpretation
     ,case
                   when upper(taken_COLOR_URINE_lab_interpretation) = 'NORMAL'  then 1
                   when upper(taken_COLOR_URINE_lab_interpretation)  = 'ABNORMAL'  then 4
                   when upper(taken_COLOR_URINE_lab_interpretation)  = 'NOT APPLICABLE'  then 0
                   when upper(taken_COLOR_URINE_lab_interpretation)  =  'VERY ABNORMAL'  then 5
                   when upper(taken_COLOR_URINE_lab_interpretation)  = 'BELOW LOW NORMAL'  then 6
                   when upper(taken_COLOR_URINE_lab_interpretation) = 'ABOVE HIGH NORMAL'  then 7
                   when upper(taken_COLOR_URINE_lab_interpretation)  like  '%NEGATIVE%'  then 2
                   when upper(taken_COLOR_URINE_lab_interpretation)  like  '%POSITIVE%'  then 3
                   else 0
                   end as taken_COLOR_URINE_lab_interpretation

       from {userName}.{algorithmName}_SummarydPatientsLabs l
   )

   group by 1,2,3
   order by 1,2,3
""")

In [18]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_labs_ml""")

DataFrame[]

In [19]:
labDf.write.saveAsTable(f"""{userName}.{algorithmName}_labs_ml""")

In [20]:
labDf.limit(100).toPandas()

,tenant,personId,encounterId,taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_test,taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test,taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test,taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test,taken_PMV_BLOOD_AUTO_lab_test,taken_HCT_BLOOD_AUTO_lab_test,taken_RDW_RATIO_AUTO_lab_test,taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_test,taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_test,taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_test,taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_test,taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_test,taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_test,taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test,...,taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation,taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_MCV_AUTO_lab_interpretation,taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation,taken_MCH_AUTO_lab_interpretation,taken_HGB_MASSVOLUME_BLOOD_lab_interpretation,taken_PROTHROMBIN_TIME_lab_interpretation,taken_COLOR_URINE_lab_interpretation
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,1,1,0,1,1,1,1,1,0,0,0,0,0,1,...,0,6,6,1,1,1,1,7,6,6,0,1,1,1,6,1,0
1,1,002756bd-6290-4982-9c69-f879158503ab,32d82cb7-69b8-4d9d-ab2d-b97e14c59927,1,1,0,1,1,1,1,1,1,1,1,1,0,1,...,1,7,7,1,1,7,6,7,7,6,0,1,1,1,1,0,0
2,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,1,1,0,1,1,1,1,1,1,1,1,1,1,1,...,6,1,1,6,1,1,1,7,6,6,0,1,1,1,1,0,0
3,1,004f0a70-f538-4e7a-b426-3b7306a59a80,ee12825a-d8a3-4e64-8de6-cd5228c8c16f,1,1,1,1,1,1,1,1,1,1,0,1,0,1,...,1,7,7,1,0,1,6,0,6,1,6,7,1,7,1,0,0
4,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,1,1,0,1,1,1,1,1,1,1,1,1,1,1,...,6,1,1,6,0,7,1,7,1,6,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0a8bf5e3-4890-46c6-8512-e85d3a821819,3f39c33d-0c59-4ac0-a8d2-aa0948c9956d,1,1,0,1,1,1,1,1,1,1,1,1,1,1,...,1,7,1,1,0,1,6,7,1,6,0,1,1,1,6,0,0
96,1,0aabc8f8-5e08-4eef-b54e-4616bfff2e7a,cdf52c4d-a161-479e-8d11-dfe095ec8e07,1,1,1,1,1,1,1,1,1,1,0,1,1,1,...,6,7,1,1,0,1,6,0,1,6,6,1,1,1,1,0,0
97,1,0ac43bfa-2789-4881-9b7e-ef0e64ebff56,2f278528-a873-4630-a8a5-685a608585c3,1,1,0,1,1,1,1,1,1,1,1,1,0,1,...,7,7,1,1,1,7,1,7,1,1,0,1,1,1,1,7,0
98,1,0aed3ee0-eeab-4e42-b89f-9c2f6a7b2954,f03d97a7-a4e0-4454-a1dc-f6a675265bf4,1,1,0,1,1,1,1,1,1,1,1,1,1,1,...,6,1,1,6,1,1,6,7,6,6,0,1,1,1,1,0,0


In [25]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsMedications""").printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- startdate: string (nullable = true)
 |-- stopdate: string (nullable = true)
 |-- taken_sodium_chloride_09percent_intravenous_solution_drugcode: string (nullable = true)
 |-- taken_propofol_drugcode: string (nullable = true)
 |-- taken_fentanyl_drugcode: string (nullable = true)
 |-- taken_insulin_lispro_drugcode: string (nullable = true)
 |-- taken_midazolam_drugcode: string (nullable = true)
 |-- taken_ceftriaxone_drugcode: string (nullable = true)
 |-- taken_potassium_chloride_drugcode: string (nullable = true)
 |-- taken_dexamethasone_drugcode: string (nullable = true)
 |-- taken_heparin_drugcode: string (nullable = true)
 |-- taken_dexmedetomidine_drugcode: string (nullable = true)
 |-- taken_tylenol_drugcode: string (nullable = true)
 |-- taken_cefepime_drugcode: string (nullable = true)
 |-- taken_acetaminophen_drugcode: string (nullable = true)
 |--

In [27]:
medicationDf = spark.sql(f"""

select m.tenant
      ,m.personId
      ,m.encounterId
,max(if(taken_sodium_chloride_09percent_intravenous_solution_drugcode is not null, 1, 0)) as taken_sodium_chloride_09percent_intravenous_solution_drugcode
,max(if(taken_propofol_drugcode is not null, 1, 0)) as taken_propofol_drugcode
,max(if(taken_fentanyl_drugcode is not null, 1, 0)) as taken_fentanyl_drugcode
,max(if(taken_insulin_lispro_drugcode is not null, 1, 0)) as taken_insulin_lispro_drugcode
,max(if(taken_midazolam_drugcode is not null, 1, 0)) as taken_midazolam_drugcode
,max(if(taken_ceftriaxone_drugcode is not null, 1, 0)) as taken_ceftriaxone_drugcode
,max(if(taken_potassium_chloride_drugcode is not null, 1, 0)) as taken_potassium_chloride_drugcode
,max(if(taken_dexamethasone_drugcode is not null, 1, 0)) as taken_dexamethasone_drugcode
,max(if(taken_heparin_drugcode is not null, 1, 0)) as taken_heparin_drugcode
,max(if(taken_dexmedetomidine_drugcode is not null, 1, 0)) as taken_dexmedetomidine_drugcode
,max(if(taken_tylenol_drugcode is not null, 1, 0)) as taken_tylenol_drugcode
,max(if(taken_cefepime_drugcode is not null, 1, 0)) as taken_cefepime_drugcode
,max(if(taken_acetaminophen_drugcode is not null, 1, 0)) as taken_acetaminophen_drugcode
,max(if(taken_azithromycin_drugcode is not null, 1, 0)) as taken_azithromycin_drugcode
,max(if(taken_piperacillin_tazobactam_drugcode is not null, 1, 0)) as taken_piperacillin_tazobactam_drugcode
,max(if(taken_lasix_drugcode is not null, 1, 0)) as taken_lasix_drugcode
,max(if(taken_insulin_aspart_drugcode is not null, 1, 0)) as taken_insulin_aspart_drugcode
,max(if(taken_remdesivir_drugcode is not null, 1, 0)) as taken_remdesivir_drugcode
,max(if(taken_norepinephrine_drugcode is not null, 1, 0)) as taken_norepinephrine_drugcode
,max(if(taken_vancomycin_drugcode is not null, 1, 0)) as taken_vancomycin_drugcode
,max(if(taken_lovenox_drugcode is not null, 1, 0)) as taken_lovenox_drugcode
,max(if(taken_morphine_drugcode is not null, 1, 0)) as taken_morphine_drugcode
,max(if(taken_zofran_drugcode is not null, 1, 0)) as taken_zofran_drugcode
,max(if(taken_magnesium_sulfate_drugcode is not null, 1, 0)) as taken_magnesium_sulfate_drugcode
,max(if(taken_dextrose_5percent_in_water_intravenous_solution_drugcode is not null, 1, 0)) as taken_dextrose_5percent_in_water_intravenous_solution_drugcode
,max(if(taken_lactated_ringers_injection_intravenous_solution_drugcode is not null, 1, 0)) as taken_lactated_ringers_injection_intravenous_solution_drugcode
,max(if(taken_glucose_drugcode is not null, 1, 0)) as taken_glucose_drugcode
,max(if(taken_ondansetron_drugcode is not null, 1, 0)) as taken_ondansetron_drugcode
,max(if(taken_lantus_drugcode is not null, 1, 0)) as taken_lantus_drugcode
,max(if(taken_insulin_regular_drugcode is not null, 1, 0)) as taken_insulin_regular_drugcode
,max(if(taken_glucagon_drugcode is not null, 1, 0)) as taken_glucagon_drugcode
,max(if(taken_aspirin_drugcode is not null, 1, 0)) as taken_aspirin_drugcode
,max(if(taken_dextrose_50percent_in_water_intravenous_solution_drugcode is not null, 1, 0)) as taken_dextrose_50percent_in_water_intravenous_solution_drugcode
,max(if(taken_enoxaparin_drugcode is not null, 1, 0)) as taken_enoxaparin_drugcode
,max(if(taken_pantoprazole_drugcode is not null, 1, 0)) as taken_pantoprazole_drugcode
,max(if(taken_furosemide_drugcode is not null, 1, 0)) as taken_furosemide_drugcode
,max(if(taken_hydralazine_drugcode is not null, 1, 0)) as taken_hydralazine_drugcode
,max(if(taken_meropenem_drugcode is not null, 1, 0)) as taken_meropenem_drugcode
,max(if(taken_albuterol_drugcode is not null, 1, 0)) as taken_albuterol_drugcode
,max(if(taken_ativan_drugcode is not null, 1, 0)) as taken_ativan_drugcode
,max(if(taken_09nacl_drugcode is not null, 1, 0)) as taken_09nacl_drugcode
,max(if(taken_rocephin_drugcode is not null, 1, 0)) as taken_rocephin_drugcode
,max(if(taken_lorazepam_drugcode is not null, 1, 0)) as taken_lorazepam_drugcode
,max(if(taken_sterile_water_drugcode is not null, 1, 0)) as taken_sterile_water_drugcode
,max(if(taken_hydromorphone_drugcode is not null, 1, 0)) as taken_hydromorphone_drugcode
,max(if(taken_atorvastatin_drugcode is not null, 1, 0)) as taken_atorvastatin_drugcode
,max(if(taken_rocuronium_drugcode is not null, 1, 0)) as taken_rocuronium_drugcode
,max(if(taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode is not null, 1, 0)) as taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode
,max(if(taken_sodium_chloride_drugcode is not null, 1, 0)) as taken_sodium_chloride_drugcode
,max(if(taken_protonix_drugcode is not null, 1, 0)) as taken_protonix_drugcode
,max(if(taken_insulin_glargine_drugcode is not null, 1, 0)) as taken_insulin_glargine_drugcode
,max(if(taken_sodium_bicarbonate_drugcode is not null, 1, 0)) as taken_sodium_bicarbonate_drugcode
,max(if(taken_oxycodone_drugcode is not null, 1, 0)) as taken_oxycodone_drugcode
,max(if(taken_zinc_sulfate_drugcode is not null, 1, 0)) as taken_zinc_sulfate_drugcode
,max(if(taken_famotidine_drugcode is not null, 1, 0)) as taken_famotidine_drugcode
,max(if(taken_amiodarone_drugcode is not null, 1, 0)) as taken_amiodarone_drugcode
,max(if(taken_sodium_chloride_09percent_injectable_solution_drugcode is not null, 1, 0)) as taken_sodium_chloride_09percent_injectable_solution_drugcode
,max(if(taken_doxycycline_drugcode is not null, 1, 0)) as taken_doxycycline_drugcode
,max(if(taken_miralax_drugcode is not null, 1, 0)) as taken_miralax_drugcode
,max(if(taken_amlodipine_drugcode is not null, 1, 0)) as taken_amlodipine_drugcode
,max(if(taken_vitamin_c_drugcode is not null, 1, 0)) as taken_vitamin_c_drugcode
,max(if(taken_dextrose_drugcode is not null, 1, 0)) as taken_dextrose_drugcode
,max(if(taken_bamlanivimab_drugcode is not null, 1, 0)) as taken_bamlanivimab_drugcode
,max(if(taken_fluvoxamine_drugcode is not null, 1, 0)) as taken_fluvoxamine_drugcode


from {userName}.{algorithmName}_SummaryPatientsMedications m
 group by 1,2,3
 order by 1,2,3

""") 

In [28]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_medications_ml""")

DataFrame[]

In [29]:
medicationDf.write.saveAsTable(f"""{userName}.{algorithmName}_medications_ml""")

In [31]:
medicationDf.limit(15).toPandas()

,tenant,personId,encounterId,taken_sodium_chloride_09percent_intravenous_solution_drugcode,taken_propofol_drugcode,taken_fentanyl_drugcode,taken_insulin_lispro_drugcode,taken_midazolam_drugcode,taken_ceftriaxone_drugcode,taken_potassium_chloride_drugcode,taken_dexamethasone_drugcode,taken_heparin_drugcode,taken_dexmedetomidine_drugcode,taken_tylenol_drugcode,taken_cefepime_drugcode,taken_acetaminophen_drugcode,taken_azithromycin_drugcode,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,0,0,0,0,0,0,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,002756bd-6290-4982-9c69-f879158503ab,32d82cb7-69b8-4d9d-ab2d-b97e14c59927,1,0,0,0,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,1,00368fc1-dd5b-4aa5-82f8-2a6c823364d3,2a36bc40-1dbe-46dd-b774-83d269d1992e,1,0,0,0,0,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,004f0a70-f538-4e7a-b426-3b7306a59a80,ee12825a-d8a3-4e64-8de6-cd5228c8c16f,1,0,0,0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1,007d5eca-d606-40d2-9031-af55a7b01dab,86b9bb4f-beb4-433a-aca3-6f32efe5d940,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
5,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,6a06b5e8-cdd5-4e98-84f4-1238237dcc8e,1,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
6,1,0092b6d2-2039-4b6b-a49f-238596cd7d36,aa061342-b172-470d-84b0-f9fe41f63beb,0,0,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,1,00a50ac0-9c00-43ba-9d25-0dcc0d23549f,757500da-0380-496f-9712-32eebe6004d7,1,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0
8,1,00a8ceb1-b0bc-40ef-9de7-6e88dc900274,ac4062df-18a5-44b1-b307-ec4bfb5908a9,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,1,00c9804e-438c-4a7c-95b3-c17718e5b4cf,29a1e91d-468f-46a9-b769-6c03b15f3c25,0,0,0,0,0,0,1,0,1,0,1,1,0,0,...,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [22]:
spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsMedications
   order by 1, 2, 3""").limit(500).toPandas().to_excel("test5.xlsx")

## Pull it all together 

In [11]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
   
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [12]:
final = spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
       
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
         and h.length_of_stay < 2*{stdValue}
              

""")

In [13]:
# drop duplicates column names
final = final.drop('tenant','personId','encounterId' )

In [14]:
# drop duplicates column names
final = final.drop('procedureId', 'serviceDate','dischargeDate','effectiveDate','startDate','stopDate','serviceStartDate','serviceEndDate' )

In [15]:
# fill in none with zeroes
final = final.fillna(0)

In [16]:
final.limit(5).toPandas()

,length_of_stay,bed_size_index,discharge_to_index,age_range,tenant_zip_code,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,12,4,3,90,5,5,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0
1,3,4,0,25,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,4,1,70,5,5,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,6,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_Final""")

DataFrame[]

In [18]:
final.write.saveAsTable(f"""{userName}.{algorithmName}_Final""")

In [19]:
final.count()

510983

In [20]:
stdVarNonVac = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVarNonVac.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [12]:
spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
     and v.vaccinated = 0
       
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
        
       
              

""").count()

544592

In [21]:
finalNonVac = spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
     and v.vaccinated = 0
       
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
         and h.length_of_stay < 2*{stdValue_nonVac}
       
              

""")

In [22]:
# drop duplicates column names
finalNonVac = finalNonVac.drop('tenant','personId','encounterId' )

In [23]:
# drop duplicates column names
finalNonVac = finalNonVac.drop('procedureId', 'serviceDate','dischargeDate','effectiveDate','startDate','stopDate','serviceStartDate','serviceEndDate' )

In [24]:
# fill in none with zeroes
finalNonVac = finalNonVac.fillna(0)

In [25]:
finalNonVac.limit(5).toPandas()

,length_of_stay,bed_size_index,discharge_to_index,age_range,tenant_zip_code,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,12,4,3,90,5,5,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0
1,3,4,0,25,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,4,1,70,5,5,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,6,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_Final_NonVac""")

DataFrame[]

In [27]:
finalNonVac.write.saveAsTable(f"""{userName}.{algorithmName}_Final_NonVac""")

In [28]:
finalNonVac.count()

510347

In [29]:
stdVarVac = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null) 
       
""")
stdValue_Vac = stdVarVac.first()['std_length_of_stay']
print(stdValue_Vac)

5.62


In [11]:
spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
     and v.vaccinated = 1
       
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
     
      
              

""").count()

641

In [30]:
finalVac = spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
     and v.vaccinated = 1
       
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
         and h.length_of_stay < 2*{stdValue_Vac}
      
              

""")

In [31]:
# drop duplicates column names
finalVac = finalVac.drop('tenant','personId','encounterId' )

In [32]:
# drop duplicates column names
finalVac = finalVac.drop('procedureId', 'serviceDate','dischargeDate','effectiveDate','startDate','stopDate','serviceStartDate','serviceEndDate' )

In [33]:
# fill in none with zeroes
finalVac = finalVac.fillna(0)

In [34]:
finalVac.limit(5).toPandas()

,length_of_stay,bed_size_index,discharge_to_index,age_range,tenant_zip_code,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,5,5,0,50,8,8,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,1,3,90,8,8,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,3,90,8,8,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,3,90,8,8,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,1,3,90,8,8,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_Final_Vac""")

DataFrame[]

In [36]:
finalVac.write.saveAsTable(f"""{userName}.{algorithmName}_Final_Vac""")

In [37]:
finalVac.count()

544

In [10]:
spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
  
           
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
         
       
              

""").count()

545233

In [38]:
finalBoth = spark.sql(f"""
select h.*,
       h.length_of_stay as label,
       v.*,  
 --      cl.*,
       c.*, 
       nc.*,
       p.*,
       l.*,
       m.*  
       
       
 from {userName}.{algorithmName}_hospital_ml h
 
 join {userName}.{algorithmName}_vaccine_ml v
    on h.tenant = v.tenant
     and h.personId = v.personId
     and h.encounterId = v.encounterId
     and ( ( v.vaccinated = 0
           and h.length_of_stay < 2*{stdValue_nonVac} )
          or  
           ( v.vaccinated = 1
           and h.length_of_stay < 2*{stdValue_Vac} ) )
           
 left join {userName}.{algorithmName}_coronavirusLabs_ml cl
    on h.tenant = cl.tenant
      and h.personId = cl.personId  
      and h.encounterId = cl.encounterId
    
 left join {userName}.{algorithmName}_conditions_ml c
    on h.tenant = c.tenant
      and h.personId = c.personId   
      and h.encounterId = c.encounterId
      
 left join {userName}.{algorithmName}_numberConditions_ml nc
    on h.tenant = nc.tenant
      and h.personId = nc.personId   
      and h.encounterId = nc.encounterId     
    
 left join {userName}.{algorithmName}_procedures_ml p
    on h.tenant = p.tenant
      and h.personId = p.personId 
      and h.encounterId = p.encounterId
    
 left join {userName}.{algorithmName}_labs_ml  l
    on  h.tenant = l.tenant
      and h.personId = l.personId 
      and h.encounterId = l.encounterId
    
 left join {userName}.{algorithmName}_medications_ml  m
    on h.tenant = m.tenant
      and h.personId = m.personId  
      and h.encounterId = m.encounterId  
 
   where h.length_of_stay > 0
         
       
              

""")

In [39]:
# drop duplicates column names
finalBoth = finalBoth.drop('tenant','personId','encounterId' )

In [40]:
# drop duplicates column names
finalBoth = finalBoth.drop('procedureId', 'serviceDate','dischargeDate','effectiveDate','startDate','stopDate','serviceStartDate','serviceEndDate' )

In [41]:
# fill in none with zeroes
finalBoth = finalBoth.fillna(0)

In [42]:
finalBoth.limit(5).toPandas()

,length_of_stay,bed_size_index,discharge_to_index,age_range,tenant_zip_code,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,intensive_care_visit,intensive_care_medical_service,palliative_care_visit,palliative_care_medical_service,hospice_care_visit,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,12,4,3,90,5,5,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0
1,3,4,0,25,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,4,1,70,5,5,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,6,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,4,0,80,5,5,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_Final_Both""")

DataFrame[]

In [44]:
finalBoth.write.saveAsTable(f"""{userName}.{algorithmName}_Final_Both""")

In [45]:
finalBoth.count()

510891